In [1]:
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch
import torchvision
from torchvision import datasets, transforms
import torch.utils.data as data
import torchvision.models as models
import matplotlib.image as pli
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from PIL import Image
from PIL import ImageOps
from PIL import ImageEnhance
import random
import math

In [2]:
label_num = 6
train_file_path = 'trainingset_image'
class ImageSet(data.Dataset):
    def __init__(self):
        self.length = 500

    def __getitem__(self, index):
        # print(index)
        label = index % label_num
        img_count = int(index / label_num) % 20 + 1
        # print(label)

        img = Image.open(f'{train_file_path}/{label}_{img_count}.jpg')
        # plt.imshow(img)
        # plt.show()
        
        new_width = min(img.size[0], img.size[1])
        img = transforms.CenterCrop(new_width)(img)
        img = img.resize((34,34))
        img = transforms.Pad(8, padding_mode='edge')(img)
        img = transforms.RandomRotation(15)(img)
        img = transforms.CenterCrop(38)(img)
        img = transforms.RandomCrop((34,34))(img)
        img = transforms.RandomHorizontalFlip(0.5)(img)
        # img = transforms.Grayscale(num_output_channels=1)(img)

        # img = transforms.Pad(12, padding_mode='edge')(img)
        # img = img.rotate(random.randint(0,15))
        # img = transforms.CenterCrop(80)(img)
        img = transforms.ToTensor()(img)
        img = transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])(img)

        img += torch.randn(3,34,34) / 1000

        # plt.imshow((img.permute(1, 2, 0).numpy() + 1) / 2)
        # print(img)
        # print(img.size())
        # plt.imshow((img[0].numpy() + 1) / 2)
        # plt.show()
        # print(aaa)
        
        return img, label

    def __len__(self):
        return self.length

In [3]:
batch_size = 256 if torch.cuda.is_available() else 8
train_loader = data.DataLoader(ImageSet(), batch_size=batch_size, shuffle=True)


In [5]:
class MiniCNN(nn.Module):
    def __init__(self):
        super(MiniCNN, self).__init__()
        # ConvLayer 1
        # 34
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels = 3,
                out_channels = 8,
                kernel_size = 3,
                stride = 1,
                padding = 0,
            ),
            # 32
            # nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
            # 16
        )
        # ConvLayer 2
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels = self.conv1[0].out_channels,
                out_channels = 16,
                kernel_size = 3,
                stride = 1,
                padding = 0,
            ),
            # 14
            # nn.BatchNorm2d(12),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
            # 7
        )
        self.avg_pool = nn.AdaptiveAvgPool2d(output_size=(1,1))
        self.fc = nn.Linear(self.conv2[0].out_channels, label_num)
    

    def forward(self, input):
        out = self.conv1(input)
        out = self.conv2(out)
        # out = self.conv3(out)
        # out = self.conv4(out)
        out = self.avg_pool(out)
        out = out.resize(out.size(0),out.size(1))
        out = self.fc(out)
        return out

print(MiniCNN)

<class '__main__.MiniCNN'>


In [630]:
convNet = MiniCNN()

In [631]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(convNet.parameters(), lr=0.01)


In [633]:
convNet.train()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
convNet = convNet.to(device)
for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    outputs = convNet(images)
    loss = loss_func(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    predict = torch.argmax(F.softmax(outputs, dim=1), dim=1)
    if i % 10 == 0:
        print(f"i = {i},  loss = {loss}, labels = {labels}, predicts = {predict},  accuracy = {float(sum(labels == predict))/float(labels.size(0))}")

i = 0,  loss = 0.5341327786445618, labels = tensor([2, 3, 4, 2, 1, 3, 3, 1]), predicts = tensor([2, 3, 2, 2, 1, 3, 3, 1]),  accuracy = 0.875
i = 10,  loss = 0.3344675898551941, labels = tensor([2, 5, 2, 0, 1, 5, 1, 1]), predicts = tensor([2, 5, 2, 0, 1, 5, 1, 1]),  accuracy = 1.0
i = 20,  loss = 0.29349228739738464, labels = tensor([2, 3, 2, 0, 0, 5, 4, 1]), predicts = tensor([2, 3, 2, 0, 0, 5, 4, 1]),  accuracy = 1.0
i = 30,  loss = 0.40547335147857666, labels = tensor([2, 2, 0, 2, 4, 0, 0, 4]), predicts = tensor([2, 2, 0, 2, 2, 0, 0, 2]),  accuracy = 0.75
i = 40,  loss = 0.20600035786628723, labels = tensor([5, 4, 2, 3, 3, 0, 2, 3]), predicts = tensor([5, 4, 2, 3, 3, 0, 2, 3]),  accuracy = 1.0
i = 50,  loss = 0.16593018174171448, labels = tensor([1, 3, 2, 2, 0, 5, 5, 2]), predicts = tensor([1, 3, 2, 4, 0, 5, 5, 2]),  accuracy = 0.875
i = 60,  loss = 0.1006302759051323, labels = tensor([3, 0, 2, 4, 3, 5, 1, 0]), predicts = tensor([3, 0, 2, 4, 3, 5, 1, 0]),  accuracy = 1.0


In [635]:
# 保存模型， 请谨慎操作， 会覆盖文件中的模型
torch.save(convNet.state_dict(), './ConvNet.model')

In [6]:
# 加载模型

convNet  = MiniCNN()
convNet.load_state_dict(torch.load('./ConvNet.model'))

<All keys matched successfully>

In [7]:
label_num = 6
file_path = 'trainingset_image'
class testSet(data.Dataset):
    def __init__(self):
        self.length = 6

    def __getitem__(self, index):
        # print(index)

        img = Image.open(f'{file_path}/{index}_21.jpg')
        plt.imshow(img)
        plt.show()
        
        new_width = min(img.size[0], img.size[1])
        img = transforms.CenterCrop(new_width)(img)
        img = img.resize((34,34))
        
        # plt.imshow(img)
        # plt.show()
        # print(img)
        # img = transforms.Grayscale(num_output_channels=1)(img)
        img = transforms.ToTensor()(img)
        img = transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])(img)

        # plt.imshow((img[0].numpy() + 1) / 2)
        print(max(img))
        print(min(img))
        plt.imshow((img.permute(1, 2, 0).numpy() + 1) / 2)
        plt.show()
        # print(img)
       
        
        return img, index

    def __len__(self):
        return self.length

test_loader = data.DataLoader(testSet(), batch_size=1, shuffle=False)

In [8]:
convNet.eval()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
convNet = convNet.to(device)
for i, (images, index) in enumerate(test_loader):
    images = images.to(device)
    labels = labels.to(device)
    outputs = convNet(images)

    predict = torch.argmax(F.softmax(outputs, dim=1), dim=1)
    
    print(f"i = {index},   predict = {predict}")

FileNotFoundError: [Errno 2] No such file or directory: 'trainingset_image/0_21.jpg'

In [9]:
print(convNet.parameters())

<generator object Module.parameters at 0x7fa3188afcf0>


In [10]:
def torch_tensor_to_cpp(t):
    return str(t).replace('[','{').replace(']','}').replace('.,',',').replace('.}','}').replace('tensor(','').replace(', dtype=torch.float64)','').replace(')','')

In [13]:
torch.set_printoptions(threshold=10000000000)
with open('layers_weight.h', 'w') as f:
    for name, data in convNet.named_parameters():
        array_str = torch_tensor_to_cpp(data.data)
        # print(array_str)
        f.write(f"""
        float layer{name.replace('.','_')}{str(data.size()).replace('torch.Size(','').replace(')','').replace(', ','][')}=
        {array_str};
        """)